<a href="https://colab.research.google.com/github/menon92/DL-Sneak-Peek/blob/master/Writting_custom_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.4.1'


## এই নোটবুকে আমরা যে বিষয় নিয়ে কথা বলব
- লেয়ার কি 
- লেয়ার ক্লাস এবং কাস্টম লেয়ার
- কাস্টম লেয়ার লেয়ার ব্যবহার করে সেলসিয়ার টু ফারেনহাইট মডেল ট্রেনিং করা
- `self.add_weight` মেথড কখন ব্যবহার করব
- `build` কখন ইমপ্লিমেন্ট করতে হবে
- কাস্টম লেয়ার লেয়ার ব্যবহার করে `MNIST` ডিজিট ক্লাসিফাই করা

## লেয়ার কি ?
লেয়ার হল যেকোনো নিউরাল নেটওয়ার্ক ডিজাইন করার বিল্ডিং ব্লক । [VGG](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png), [ResNet](https://www.researchgate.net/profile/Weizhen-Fang-2/publication/335213445/figure/fig2/AS:793426240995328@1566178965482/Structure-of-the-ResNet-50-used-for-reservoir-recognition.jpg) এই নেটওয়ার্ক গুলো আসলে বিভিন্ন লেয়ার এর সমন্বয়ে গঠিত । নেটওয়ার্ক ডিজাইনের সুবিধার্থে টেন্সরফ্লো যে লেয়ার গুল খুব বেশি ব্যবহার হয় সেগুলো আগে থেকেই ইমপ্লিমেন্ট করে রেখেছে । যেমন : [Dense](https://keras.io/api/layers/core_layers/dense/), [Conv1D](https://keras.io/api/layers/convolution_layers/convolution1d), [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d). এই রকম আরও [অনেক](https://keras.io/api/layers/) প্রয়োজনীয় লেয়ার ইমপ্লিমেন্ট করাই আছে । আপনি আপানর প্রয়োজন অনুসারে এইগুলো ব্যবহার করতে পারেন । 

এই সবগুলো লেয়ারের মুল কাজ এর কাছে আসা ইনপুট কে ট্রান্সফরম করা এবং এই ট্রান্সফরমড অউতপুত রিটার্ন করা । ডাটার এই ট্রান্সফরমেশন একেক লেয়ার একেক ভাবে হয়ে থেকে । 

অনেক সময় যদি আপনার এমন দরকার হয় যে আপনি আপনার মত করে ইনপুট ডাটার ট্রান্সফরমাশন করতে চান তাহলে কাস্টম লেয়ার ইমপ্লিমেন্ট করতে পারেন । 

লেয়ার ক্লাস গুল ওয়েট ভেরিয়াব, বায়াস ভেরিয়াবল এবং কিছু কম্পুউটেসনের সমন্বয়ে গঠিত হয় । মনে করে আমরা একটা কাস্টম লেয়র লিখতে চাই যেটা দেখতে নিচের মত । 

```
y = W*x + b
```
যেখানে,
```
লেয়ার ওয়েট ভেরিয়েবল
লেয়ার বায়াস ভেরিয়েবল 
লেয়ারের ইনপুট
লেয়ারের অউতপুট
```
ওয়েট, বায়াস কে লেয়ারের স্টেট ভেরিয়েবলও বলা হয়ে থাকে । এই রকম নাম দেয়ার করন আপনি যখন এই লেয়ার কে নেটওয়ার্ক ডিজাইন করার সময় ব্যবহার করবেন এবং আপনার নেটওয়ার্ক কে কোন ডাটার উপর ট্রেনিং করবেন তখন প্রতিবার নেটওয়ার্ক [ব্যাক-প্রপ্রাগ্রেশনের](https://en.wikipedia.org/wiki/Backpropagation) সময় এই সব স্টেট ভেরিয়েব কে আপডেট করে । অর্থাৎ আপনার পুরার নেটওয়ার্ক এর স্টেট পরিবর্তন হয় । 

## লেয়ার ক্লাস এবং কাস্টম লেয়ার

কাস্টম লেয়ার তৈরি করর জন্য আমাদের কয়েকটা জিনিস লাগবে, 

- Layer ক্লাস ইনহেরিট করা 
- __init__() লেয়ারের প্রয়োজনীয় ভেরিয়াবল ইনিসিয়ালাইজ করা
- call() ইনপুট ডাটা ট্রান্সফরমেসন লজিক ইমপ্লিমেন্ট করা
- build() যদি আমাদের এমন কোন ভেরিয়েবল ইনিসিয়ালাইজ করর দরকার হয় সেটা ইনপুট সেইপ আগে থেকে জানি না । সেই ধরনের ভেরিয়েবল আমাদের কে build এর মাঝে ইনিসিয়ালাজ করতে হবে 



In [ ]:
class Linear(keras.layers.Layer): # লেয়ার ক্লাস কে ইনহেরিট করি
    '''Custom layer that implements y = wx + b
    '''
    def __init__(self, units=8, input_dim=8):
        # মুল Layer ক্লাস কে ইনিসিলাইজ করি
        super(Linear, self).__init__()

        # tf.random_normal_initializer() ফাংশন কোন ডিরেক্ট রেন্ডম নাম্বার জেনারেট করে না 
        # শুধু রেন্ডম নাম্বার জেনারেট করার জন্য ইনিসিয়ালাইজ হয়ে থাকে, এটা কে যেকোনো শেপ দিয়ে কল করলে 
        # এটা সেই শেপের রেন্ডম নাম্বার জেনারেট করে দেয়। 
        w_init = tf.random_normal_initializer()
        print(f"w_init: {w_init}")

        # আমরা এখানে `w_init` ব্যবহার করে w কে রেন্ডম নাম্বার দিয়ে ইনিসিয়ালাইজ করি । 
        self.w = tf.Variable(
            # আমরা সেপ ডাটা টাইপ দিয়ে দিলাম
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            # ব্যাক-প্রপাগ্রেসনের সময় w এর মান পরিবর্তন হবে কি না সেটা বলে দিলাম। অনেক সময় ট্রান্সফার লার্নিং 
            # করার সময় দরকার হয় যে, আমরা আগের ট্রেনিং করা ওয়েট কে পরিবর্তন করব না । তখন আমরা 
            # এটা কে False করে দিলেই ব্যাক-প্রপাগ্রেসনের সময় w পরিবর্তন হবে না ।
            trainable=True
        )
        print(f'self.w: {self.w}')
        # ওয়েট ভেরিয়েবল এর মত একই ভাবে আমরা বায়াস ভেরিয়েবল কে ইনিসিয়ালিজ করি
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"),
            trainable=True
        )
        print(f'b_init: {b_init}')
        print(f'self.b: {self.b}')

    def call(self, x):
        # w*x + b  ট্রান্সফরমেসন লজিক ইমপ্লিমেন্ট করি
        return tf.matmul(x, self.w) + self.b

এখন আমরা আমাদের বানানো কাস্টম লেয়ার কে স্যাম্পল ডাটা দিয়ে টেস্ট করে দেখব। 

In [ ]:
# 1x4 সাইজের ইনপুট ডিফাইন করি । এটা কে আমরা লেয়ারে ইনপুট ডাটা হিসাবে ব্যবহার করব
# এখানে আমাদের ১ টা ডাটা আছে যার ডাইমেসন ৪
x = tf.ones((1, 4))

# লেয়ার ইনিসিয়ালাইজ করি 
linear_layer = Linear(
    # এই লেয়ারে ডাটা উপর ট্রান্সফরমেসন অ্যাপ্লাই হওয়ার পরে 
    # যে ডাটা আমরা পাব তার শেষ ডাইমেসন কত চাচ্ছি সেটা ডিফাইন করে দিলাম
    # units = 2 মানের লাস্ট ডাইমেসন হবে 2, 16 দিলে 16 হবে 
    units=2,
    # যেহেতু আমাদের ডাটার ডাইমেনসন 4 তাই আমরা input_dim = 4 সেট করে দিলাম ।
    # অন্য ডাইম্যানসন দিলে এরর হবে
    input_dim=4
)

# আমাদের ডাটা x কে আমাদের বানানো লেয়ারের মাঝে দিয়ে পাস করি
y = linear_layer(x)

# লেয়ার ডাটা x কে ট্রান্সফরম করার পরের অউতপুট প্রিন্ট করি
print(f"\ny :{y}")
print(f"shape: {y.shape}")

w_init: <tensorflow.python.ops.init_ops_v2.RandomNormal object at 0x7f1e295a7c50>
self.w: <tf.Variable 'Variable:0' shape=(4, 2) dtype=float32, numpy=
array([[-0.02637166,  0.06013067],
       [ 0.05447754,  0.05503648],
       [-0.01913537, -0.01881706],
       [-0.07917801,  0.01136541]], dtype=float32)>
b_init: <tensorflow.python.ops.init_ops_v2.Zeros object at 0x7f1e295a7a90>
self.b: <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

y :[[-0.07020751  0.10771549]]
shape: (1, 2)


শেপের মান আমরা পাইলাম shape: (1, 2) অর্থাৎ লাস্ট ডাইমেনসন 2 কারণ আমরা ইউনিট = 2 দিয়েছিলাম


## কাস্টম লেয়ার লেয়ার ব্যবহার করে সেলসিয়ার টু ফারেনহাইট মডেল ট্রেনিং করা
এখন আমাদের বানানো এই কাস্টম লেয়ার ব্যবহার করে কোন তাপমাত্রা সেলসিয়াস এ দেয়া থাকলে সেটা ফারেনহাইটে কনভার্ট করতে পারি কি না সেটা চেক করে দেখব। সেলসিয়াস থেকে ফারেনহাইটে কনভার্ট করার সূত্র এই রকম,  

# $$ f = c \times 1.8 + 32 $$

এখানে `self.w = 1.8, self.b = 32` হিসাবে চিন্তা করতে পারি । নিচে আমরা দেখব আমাদের কাস্টম লেয়ার এই মান গুলো শিখতে পারে কি না । 


In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=8, input_dim=8):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"),
            trainable=True
        )
    def call(self, x):
        return tf.matmul(x, self.w) + self.b

# সেলসিয়াস এ তাপমাত্রা
X = np.array([[-40], [-10],  [0],  [8], [15], [22],  [38]],  dtype='float32')
# ফারেনহাইট এ তাপমাত্রা 
y = np.array([[-40],  [14], [32], [46], [59], [72], [100]],  dtype='float32')

# Loss function
def loss(real_y, pred_y):
    return tf.abs(real_y - pred_y)

epochs = 501
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
# লেয়ার ইনিসিয়ালাইজ করি
linear_layer = Linear(units=1, input_dim=1)

def train_step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # take transformed output from layer
        pred_y = linear_layer(real_x)
        # calculate loss
        reg_loss = loss(real_y, pred_y)
    # Calculate gradients with respect to reg_loss
    grads = tape.gradient(reg_loss, linear_layer.trainable_weights)
    # Update trainable variables
    optimizer.apply_gradients(zip(grads, linear_layer.trainable_weights))

    return reg_loss

# training the layer up to given epochs
for e in range(epochs):
    reg_loss = train_step(X, y)
    if e % 100 == 0:
        # show loss, status of state variable w, b
        updated_w, updated_b = linear_layer.trainable_weights
        print(f'Epoch: {e} loss: {tf.reduce_sum(reg_loss).numpy():.3f}')
        print(
            f'Updated w: {np.squeeze(updated_w.numpy()):.3f} '
            f'updated b: {np.squeeze(updated_b.numpy()):.3f}'
        )
        print('-' * 40)

Epoch: 0 loss: 361.588
Updated w: 0.112 updated b: 0.100
----------------------------------------
Epoch: 100 loss: 134.785
Updated w: 2.362 updated b: 10.119
----------------------------------------
Epoch: 200 loss: 74.673
Updated w: 2.113 updated b: 20.014
----------------------------------------
Epoch: 300 loss: 14.678
Updated w: 1.863 updated b: 29.902
----------------------------------------
Epoch: 400 loss: 1.267
Updated w: 1.815 updated b: 31.998
----------------------------------------
Epoch: 500 loss: 1.572
Updated w: 1.788 updated b: 31.993
----------------------------------------


In [ ]:
# ট্রেনিং শেষ এখন আমরা ডাটা দিয়ে টেস্ট করে দেখি আমদের লেয়ার কিছু শিখতে পারল কি না 
prediction = linear_layer(X)
for pred, target in zip(prediction, y):
    print(f"Target fahrenheit: {target} predicted: {pred}")

Target fahrenheit: [-40.] predicted: [-39.536186]
Target fahrenheit: [14.] predicted: [14.111048]
Target fahrenheit: [32.] predicted: [31.99346]
Target fahrenheit: [46.] predicted: [46.29939]
Target fahrenheit: [59.] predicted: [58.817078]
Target fahrenheit: [72.] predicted: [71.33476]
Target fahrenheit: [100.] predicted: [99.946625]


উপরে আমরা দেখতে পাচ্ছি যে মোটামুটি ঠিক ভাবেই সেলসিয়াস কে ফারেনহাইটে পরিবর্তন করতে পারছে । আমরা এই লেয়ার কে `[-40], [-10],  [0],  [8], [15], [22],  [38]` এই কয়েকটা ডাটা দিয়ে ট্রেনিং করাইছি । এখন দেখি আমরা যদি এর বাহিরের কোন ডাটা ইনপুট দেয় তাহলে কি হয় । 

In [ ]:
celsius = 100
print(
    'Predicted:', np.squeeze(linear_layer(np.array([[celsius]], dtype='float32'))), 
    'actual:', 1.8*celsius + 32
)

Predicted: 210.81757 actual: 212.0


নতুন ডাটার জন্য লেয়ার কাছাকাছি মানই দিচ্ছে । এই রকম হওয়ার কারণ হল আমরা যদি ৫০০ ইপকে ট্রেনিং লগ দেখি সেটা কিছুটা এই রককম, 

```
Epoch: 500 loss: 1.890
Updated w: 1.795 updated b: 31.993
```
অর্থাৎ `w, b` এর মান যথাক্রম `1.795, 31.993` যেটা  `1.8, 32` এর কাছাকাছি । 


## কখন এবং কেন build() ব্যবহার করব 

আমরা শুরুর দিকে বলেছিলাম যে build তখনি ইমপ্লিমেন্ট করতে হবে যদি আমরা আগে থেকে লেয়াররের ইনপুট কি হবে সেটা বলে দিতে না পারি । আমাদের যদি পুরা নেটওয়ার্কের জন্য কেবল মাত্র ১ তা লেয়ার ব্যবহার করি তাহলে build এর দরকার নাই. 

কিন্তু রিয়েল নেটওয়ার্ক গুল অনেক লেয়ার নিয়ে গঠিত হয় কিছুটা এই রকম

```python
# l1 এর input_dim আমরা দিতে পারতেছি
l1 = Linear(units=8, input_dim=100)
# কিন্তু l2 এর input_dim নির্ভর করতেছে l1 এর অউতপুটের উপর
l2 = Linear(units=16)(l1)
l3 = Linear(units=16)(l2)
# একই ভাবে l4 এর input_dim নির্ভর করতেছে l3 এর উপর
l4 = Linear(units=32)(l3)
```

উপরের বিষয়টা যদি আমরা হ্যান্ডলে করতে চাই তাহলে আমাদের কে `build` ইমপ্লিমেন্ট করতে হবে ।  

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        # এখানে আমরা input_shape দিলাম না । 
        self.units = units

    def build(self, input_shape):
        # w_init = tf.random_normal_initializer()
        # self.w = tf.Variable(
        #     initial_value=w_init(shape=(input_dim, units), dtype="float32"),
        #     trainable=True
        # )
        # উপরে যেভাবে ওয়েট ইনিসিয়ালাইজ করা হয়েছে সেটা ১ লাইনে করতে চাইলে আমরা 
        # `keras.layers.Layer` এর  self.add_weight মেথড ব্যবহার করতে পারি  
        self.w = self.add_weight(
            # ইনপুট ডাটার ডাইম্যানসনের উপর নির্ভর করে ইনপুট সেপ নিয়ে নিলাম
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        x = tf.matmul(inputs, self.w) + self.b
        # এই লেয়ার আমরা MNIST ডাটাসেটের উপর ব্যবহার করব, যেহেতু এই ডাটা 
        # নিনিয়ার ভাবে ক্লাসিফাই করা যায় না এই জন্য আমরা লেয়ারে নন-লিনিয়ারিটি
        # অ্যাড করলাম relu মেথডের মাধ্যমে 
        return tf.nn.relu(x)

In [ ]:
# কিছু স্যাম্পল ডাটা তৈরি করে ৩ লেয়েয়ার মধ্যে দিয়ে ডাটা 
# পার করে দেখি কেমন অউতপুট দেয়

inputs = tf.ones([1, 2])
print('Sample input:', inputs)

l1 = Linear(8)(inputs)
l2 = Linear(8)(l1)
l3 = Linear(4)(l2)

print('Transformed output after passing it all the three layer:')
print(l3)

Sample input: tf.Tensor([[1. 1.]], shape=(1, 2), dtype=float32)
Transformed output after passing it all the three layer:
tf.Tensor([[0.01034534 0.         0.01573105 0.02405714]], shape=(1, 4), dtype=float32)


## কাস্টম লেয়ার লেয়ার ব্যবহার করে MNIST ডিজিট ক্লাসিফাই করা

আমাদের লেয়ার এখন ইনপুট ডাইমেনসন বলে না দিলেও কাজ করে । এই জন্য আমরা একাধিক লেয়ার সমন্বয় করে `MNIST` ডাটাসেট ক্লাসিফাই করার নেটওয়ার্ক ডিজাইন করে ফেলি।  

In [ ]:
# একাধিক লেয়ার দিয়ে নেটওয়ার্ক ডিফাইন করি 
model = keras.Sequential([
    keras.Input(shape=(784,)),
    Linear(64),
    Linear(64),
    Linear(10),
])

# optimizer.apply_gradients() এর মাধ্যমে আমদের কাস্টম লেয়ার গুলোর w, b উপডেট করবে
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
# নেটওয়ার্কের লস হিসাব করার জন্য ব্যবহার হবে
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# ট্রেনিং আকুরেসি ট্রাক করার কাজে এটা ব্যবহার হবে
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
# ভ্যালিডেসন আকুরেসি ট্রাক করার কাজে এটা ব্যবহার হবে
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

# ইন্টারনেট থেকে MNIST ডাটা ডাউনলোড করি
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# spilt data into train, valication
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
import time

epochs = 5
for epoch in range(epochs):
    start_time = time.time()

    # iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # নেটওয়ার্ক থেকে অউতপুট নেই
            logits = model(x_batch_train)
            # নেটওয়ার্কের অউতপুট এবং অরিজিনাল লেবেল থেকে লস হিসাব করি
            loss_value = loss_fn(y_batch_train, logits)
        # লস এর সাপেক্ষে নেটওয়ার্ক এর সব w, b এর গ্র্যাডিয়েন্ট হিসাব করি
        grads = tape.gradient(loss_value, model.trainable_weights)
        # এই গ্র্যাডিয়েন্ট এবং অপটিমাইজার ব্যবহার করে w, b এর মান উপডেট করি
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        # update training metric.
        train_acc_metric.update_state(y_batch_train, logits)
    # store train metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    # reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # perform validatoin
    # run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val)
        # update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    # store current validation accuracy
    val_acc = val_acc_metric.result()
    # reset the validation metrics at the end of each epoch
    val_acc_metric.reset_states()

    # display training, validaton summary
    print(
        f"Epoch: {epoch+1} - train acc: {float(train_acc):.3f} - "
        f"val acc: {float(val_acc):.4f} - time: {time.time() - start_time:.2f}s"
    )

Epoch: 1 - train acc: 0.357 - val acc: 0.5712 - time: 7.87s
Epoch: 2 - train acc: 0.645 - val acc: 0.6710 - time: 7.77s
Epoch: 3 - train acc: 0.745 - val acc: 0.7687 - time: 7.87s
Epoch: 4 - train acc: 0.774 - val acc: 0.7714 - time: 7.76s
Epoch: 5 - train acc: 0.777 - val acc: 0.7740 - time: 7.74s


## Referance
- [Custom layer keras](https://www.tensorflow.org/tutorials/customization/custom_layers)
- [Guide to custom layer](https://www.tensorflow.org/guide/keras/custom_layers_and_models)
- [New layer creation using subclassing](https://keras.io/guides/making_new_layers_and_models_via_subclassing/)